### Install Vertex AI SDK

In [1]:
# Remove remark to install packages
# !pip install -U google-cloud-aiplatform --user

In [2]:
# Remove remark to authenticate
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

In [3]:
from vertexai.preview.language_models import TextGenerationModel,\
                                            ChatModel,\
                                            InputOutputTextPair,\
                                            TextEmbeddingModel

from IPython.display import display, Markdown
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

PROJECT_ID = "jerry-argolis"
LOCATION = "us-central1"

# Initialize Vertex AI SDK
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

2023-09-11 18:48:32.759230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Chat model with `chat-bison@001`

The `chat-bison@001` model lets you have a freeform conversation across multiple turns. The application tracks what was previously said in the conversation. As such, if you expect to use conversations in your application, use the `chat-bison@001` model because it has been fine-tuned for multi-turn conversation use cases.

In [4]:
chat_model = ChatModel.from_pretrained("chat-bison@001")
print(type(chat_model))

chat = chat_model.start_chat()
print(type(chat))

print(chat.send_message("""
My name is Larry.
"""))

<class 'vertexai.preview.language_models._PreviewChatModel'>
<class 'vertexai.language_models.ChatSession'>
Hi Larry, how can I help you today?


As shown below, the model should respond based on what was previously said in the conversation:

In [5]:
print(chat.send_message("""
What's my name again?
"""))

Your name is Larry.


In [7]:
print(type(chat))

<class 'vertexai.language_models.ChatSession'>


In [8]:
# print the chat history
for message in chat.message_history:
    # print(message)
    print(message.author, ':', message.content)



user : 
My name is Larry.

bot : Hi Larry, how can I help you today?
user : 
What's my name again?

bot : Your name is Larry.


### Advanced Chat model with the SDK
You can also provide a `context` and `examples` to the model. The model will then respond based on the provided context and examples. You can also use `temperature`, `max_output_tokens`, `top_p`, and `top_k`. These parameters should be used when you start your chat with `chat_model.start_chat()`.

For more information on chat models, please refer to the [documentation on chat model parameters](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#chat_model_parameters).

In [9]:
chat = chat_model.start_chat(
    context="My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.",
    examples=[
        InputOutputTextPair(
            input_text="Who do you work for?",
            output_text="I work for Ned.",
        ),
        InputOutputTextPair(
            input_text="What do I like?",
            output_text="Ned likes watching movies.",
        ),
    ],
    temperature=0.3,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=1
)
print(chat.send_message("Are my favorite movies based on a book series?"))

Yes, your favorite movies are based on a book series.


In [10]:
print(chat.send_message("When where these books published?"))

The books were published in 1954 and 1955.


In [11]:
# print the chat history
print(chat.__dict__)
print(chat._context)
print(chat._examples)
print(chat._temperature)
for message in chat.message_history:
    # print(message)
    print(message.author, ':', message.content)

{'_model': <vertexai.preview.language_models._PreviewChatModel object at 0x154149ed0>, '_context': 'My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.', '_examples': [InputOutputTextPair(input_text='Who do you work for?', output_text='I work for Ned.'), InputOutputTextPair(input_text='What do I like?', output_text='Ned likes watching movies.')], '_max_output_tokens': 1024, '_temperature': 0.3, '_top_k': 1, '_top_p': 0.8, '_message_history': [ChatMessage(content='Are my favorite movies based on a book series?', author='user'), ChatMessage(content='Yes, your favorite movies are based on a book series.', author='bot'), ChatMessage(content='When where these books published?', author='user'), ChatMessage(content='The books were published in 1954 and 1955.', author='bot')], '_stop_sequences': None}
My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.
[InputOutputTextPair(input_text='Who do you wor

* message_history에 대화 내용이 저장되고
* 그 중에서 max input token을 넘지 않는 범위 내에서 input token으로 파라미터화 하여 send_message() 호출
* max input token(4096)을 넘어서면, 오랜 history 대화는 input token으로 전달되지 않을 수 있음

In [55]:
def count_tokens(chat):
    count = 0
    for message in chat.message_history: 
        count += len(message.content.split())
    return count

chat = chat_model.start_chat(
    context="My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.",
    examples=[
        InputOutputTextPair(
            input_text="Can you summarize this for me: \n The history of the conversation as a list of messages. Code chat does not support context.",
            output_text="Sure, let me summarize it for you: Code chat does not support context.",
        )
    ],
    temperature=0.3,
    max_output_tokens=50,
    top_p=0.8,
    top_k=3
)

# read text from the url
import requests
url = "https://rainbowfluffysheep.wordpress.com/the-longest-text-ever/"
r = requests.get(url)
text = r.text
print(len(text), len(text.split()))

# create more than 4096 tokens in msg variable
print(chat.send_message("My name is Jerry."))
print(count_tokens(chat))
print("CONTEXT 0: ", chat.send_message("What's my name?"))
# msg = "I am a very long message. " * 1000
msg = "Can you summarize this for me: \n{}".format(text[:5000])
print("RESPONSE 1: ", chat.send_message(msg))
print(count_tokens(chat))
print("CONTEXT 1: ", chat.send_message("What's my name?"))
# msg = "I am the second very long message. " * 1000
msg = "Can you summarize this for me: \n{}".format(text[5000:20000])
print("RESPONSE 2: ", chat.send_message(msg))
print(count_tokens(chat))
print("CONTEXT 2: ", chat.send_message("What's my name?"))
# msg = "I am the third very long message. " * 1000
msg = "Can you summarize this for me: \n{}".format(text[20000:100000])
print("RESPONSE 3: ", chat.send_message(msg))
print(count_tokens(chat))
print("CONTEXT 3: ", chat.send_message("What's my name?"))
# msg = "I am the forth very long message. " * 1000
msg = "Can you summarize this for me: \n{}".format(text[100000:])
print("RESPONSE 4: ", chat.send_message(msg))
print(count_tokens(chat))
print("CONTEXT 4: ", chat.send_message("What's my name?"))
# print(chat.message_history)


215858 30429
Nice to meet you Jerry. What can I help you with?
15
CONTEXT 0:  Your name is Jerry.
RESPONSE 1:  Sure, let me summarize it for you: The longest text ever is a text that is so long that it is impossible to read in one sitting. It is often used as a test of endurance or as a way to show off one's typing
329
CONTEXT 1:  Your name is Jerry.
RESPONSE 2:  OK, I'll summarize the following for you:

The history of the conversation as a list of messages. Code chat does not support context.
835
CONTEXT 2:  Your name is Jerry.
RESPONSE 3:  I'm not able to help with that, as I'm only a language model. If you believe this is an error, please send us your feedback.
12550
CONTEXT 3:  I don't know your name. Can you tell me?
RESPONSE 4:  I'm not able to help with that, as I'm only a language model. If you believe this is an error, please send us your feedback.
30621
CONTEXT 4:  I don't know your name.


* message history를 외부(예: DB 등)에서 가져와 사용하는 방식
* ChatSession을 매번 생성

In [12]:
# DB에서 대화 내용을 가져온다.
messages = [
    {
        "author": "user", 
        "content": "My name is Jerry."
    },
    {
        "author": "bot", 
        "content": "Hi Jerry, I am your personal assistant."
    },
    {
        "author": "user",
        "content": "What's my name?"
    },
    {
        "author": "bot",
        "content": "Your name is Jerry."
    },
]


In [17]:
from vertexai.language_models import ChatMessage, InputOutputTextPair

vertex_messages = []
for m in messages:
    vertex_message = ChatMessage(content=m["content"], author=m["author"])
    vertex_messages.append(vertex_message)

chat = chat_model.start_chat(
    message_history=vertex_messages,
    temperature=0.3,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=1
)

print(chat.send_message("What's my name?"))
    

Your name is Jerry.
